In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import scipy.io as sio
import anndata as ad
import seaborn as sns
import os as os
import sys as sys
sys.path.append('/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/')
from scRNA_utils import *
import operator as op
import matplotlib.colors as mcolors

In [4]:
results = pd.read_excel('/home/qiuaodon/Desktop/project_data_new/grouped_CD8EX_GEM_0927.xlsx')


In [11]:
lr_pairs = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/lr_network_unique.tsv',  sep='\t')

lr_pairs.columns = ['L', 'R', 'LR_source', 'database' ]

In [13]:
# add source and database to the results which rows match the L and R
results['LR_source'] = ''
results['database'] = ''
for i in range(len(lr_pairs)):
    for j in range(len(results)):
        if lr_pairs.iloc[i,0] == results.iloc[j,1] and lr_pairs.iloc[i,1] == results.iloc[j,2]:
            results.iloc[j,16] = lr_pairs.iloc[i,2]
            results.iloc[j,17] = lr_pairs.iloc[i,3]

results

,Unnamed: 0,L,R,GEM_1 (X),GEM_2 (Y),g1,g2,Num_genes_g1,Num_genes_g2,Corr (X-Y),Corr (cond.),Corr (X-L),Corr (R-Y),Corr (X-LR),Corr (LR-Y),p-value (CIT fisherz),LR_source,database
0,Mono,ANXA1,C5AR1,GEM_0_CD8EX,GEM_0_Mono,"['NFKBIA', 'TSC22D3', 'TXNIP']","['PDK4', 'PFKFB3', 'RCAN1', 'SESN1', 'VEGFA']",3,5,0.535098,0.369079,0.566059,0.552606,0.532493,0.495198,0.003178,ppi_lr,ppi_prediction
1,Mono,ANXA1,CCR7,GEM_1_CD8EX,GEM_1_Mono,"['GLIPR1', 'NFKBIA', 'TXNIP']","['AREG', 'GPR183', 'GRASP']",3,3,0.552793,0.427753,0.533045,0.394872,0.502856,0.438488,0.000499,ppi_lr,ppi_prediction
2,Mono,ANXA1,CXCR4,GEM_2_CD8EX,GEM_2_Mono,"['FABP5', 'GLIPR1', 'NFKBIA', 'TSC22D3', 'TXNIP']","['ETS2', 'GPR183', 'GRASP', 'HMGB2', 'PDK4', '...",5,9,0.597621,0.422257,0.570680,0.578289,0.666807,0.483029,0.000603,ppi_lr,ppi_prediction
3,Mono,CALR,LRP5,GEM_3_CD8EX,GEM_3_Mono,"['FABP5', 'NFKBIA', 'TSC22D3']","['ETS2', 'PDK4', 'RCAN1', 'SESN1', 'TSC22D1', ...",3,7,0.531898,0.357473,0.445511,0.622797,0.479307,0.573505,0.004397,ppi_lr,ppi_prediction
4,Mono,CCL5,CXCR4,GEM_4_CD8EX,GEM_4_Mono,"['NFKBIA', 'TSC22D3', 'TXNIP']","['GPR183', 'HMGB2', 'PKP2', 'RNF144B', 'SESN1']",3,5,0.640765,0.521636,0.423427,0.572969,0.540172,0.468067,0.000011,ppi_lr,ppi_prediction
5,Mono,RPS19,C5AR1,GEM_5_CD8EX,GEM_5_Mono,"['FABP5', 'NFKBIA', 'TSC22D3', 'TXNIP']","['PDK4', 'RCAN1', 'SESN1', 'TSC22D1', 'VEGFA']",4,5,0.540741,0.359856,0.651873,0.544546,0.557856,0.508260,0.004117,pharmacology,guide2pharmacology
6,B,ITGA4,JAM3,GEM_6_CD8EX,GEM_0_B,"['TSC22D3', 'TXNIP']","['AC025164.1', 'KLF9']",2,2,0.356992,0.266032,0.445108,0.485123,0.519229,0.265360,0.037897,ppi_bidir_bidir,ppi_prediction
7,B,ITGB1,JAM3,GEM_6_CD8EX,GEM_0_B,"['TSC22D3', 'TXNIP']","['AC025164.1', 'KLF9']",2,2,0.356992,0.232519,0.438506,0.485123,0.499022,0.334915,0.071262,ppi_bidir_bidir,ppi_prediction
8,DC,ANXA1,CXCR4,GEM_7_CD8EX,GEM_0_DC,"['NFKBIA', 'TSC22D3']","['DDIT4', 'ERN1', 'GRASP', 'IRF4', 'RNF144B', ...",2,8,0.422215,0.200971,0.503287,0.832320,0.486664,0.571547,0.130787,ppi_lr,ppi_prediction
9,DC,ITGA4,CXCR4,GEM_7_CD8EX,GEM_1_DC,"['NFKBIA', 'TSC22D3']","['DDIT4', 'ERN1', 'RNF144B', 'TSC22D3', 'ZFP36...",2,5,0.396728,0.221952,0.400176,0.846269,0.431766,0.523906,0.094151,ppi_bidir_r,ppi_prediction


In [14]:
# save the results
results.to_excel('/home/qiuaodon/Desktop/project_data_new/grouped_CD8EX_GEM_0927.xlsx', index = False)